In [18]:
import json

# 假设JSON数据已经保存在变量中，如果是从文件读取，可以使用以下代码：
with open('宋词_pretrain.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def extract_text_from_json(data):
    result = []
    for poem in data:
        # 添加词牌名
        result.append(poem["rhythmic"])
        
        # 添加所有句子
        for sentence in poem["paragraphs"]:
            result.append(sentence)
        
        # 添加分隔符
        result.append("<|endoftext|>")
    
    return "\n".join(result)

# 提取文本
extracted_text = extract_text_from_json(data)
# print(extracted_text)

# 如果需要保存到文件
with open('pretrain.txt', 'w', encoding='utf-8') as f:
    f.write(extracted_text)

In [19]:
with open('pretrain.txt', 'r', encoding='utf-8') as f:
    content = f.read()

# 移除所有 <|endoftext|>
content = content.replace('<|endoftext|>', '')

with open('pretrain.txt', 'w', encoding='utf-8') as f:
    f.write(content)

In [20]:
import json
from pathlib import Path

def convert_ci_data_to_sft_format(input_file, output_file):
    """
    将宋词JSON数据转换为SFT训练格式
    
    Args:
        input_file: 输入的JSON文件路径
        output_file: 输出的JSONL文件路径
    """
    # 读取原始数据
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 准备输出数据
    sft_data = []
    
    for item in data:
        # 构建instruction：词牌名
        instruction = f"{item['rhythmic']}"
        
        # 构建output：将所有段落用换行符连接，并添加<eos>标记
        paragraphs = item['paragraphs']
        output_text = "\n".join(paragraphs) + "\n"
        
        # 创建SFT格式的记录
        sft_record = {
            "instruction": instruction,
            "input": "",  # 输入为空
            "output": output_text
        }
        
        sft_data.append(sft_record)
    
    # 保存为JSONL格式（每行一个JSON对象）
    with open(output_file, 'w', encoding='utf-8') as f:
        for record in sft_data:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
    
    print(f"转换完成！共处理 {len(sft_data)} 条数据")
    print(f"输出文件: {output_file}")     


def preview_sft_data(output_file, num_samples=2):
    """预览生成的SFT数据"""
    print("\n预览生成的SFT数据：")
    print("=" * 50)
    
    with open(output_file, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= num_samples:
                break
            record = json.loads(line.strip())
            print(f"样本 {i+1}:")
            print(f"Instruction: {record['instruction']}")
            print(f"Input: {record['input']}")
            print(f"Output: {record['output'][:100]}...")  # 只显示前100个字符
            print("-" * 50)

def main():
    # 输入文件路径（假设您的数据保存在这个文件）
    input_file = "宋词_SFT.json"  # 修改为您的实际文件路径
    
    # 输出文件路径（符合原代码要求的路径）
    output_file = "宋词_SFT.jsonl"
    
    # 检查输入文件是否存在
    if not Path(input_file).exists():
        print(f"错误：输入文件 {input_file} 不存在！")
        print("请将您的JSON数据保存为 ci_data.json 或修改 input_file 变量")
        return
    
    # 使用版本1进行转换（最接近您的要求）
    print("正在转换数据...")
    convert_ci_data_to_sft_format(input_file, output_file)
    
    # 预览生成的数据
    preview_sft_data(output_file)

if __name__ == "__main__":
    main()


正在转换数据...
转换完成！共处理 280 条数据
输出文件: 宋词_SFT.jsonl

预览生成的SFT数据：
样本 1:
Instruction: 湘春夜月
Input: 
Output: 近清明。
翠禽枝上消魂。
可惜一片清歌，都付与黄昏。
欲共柳花低诉，怕柳花轻薄，不解伤春。
念楚乡旅宿，柔情别绪，谁与温存。
空樽夜泣，青山不语，残月当门。
翠玉楼前，惟是有、一波湘水，摇荡湘云。
天...
--------------------------------------------------
样本 2:
Instruction: 瑞鹤仙
Input: 
Output: 湿云黏雁影。
望征路愁迷，离绪难整。
千金买光景。
但疏钟催晓，乱鸦啼暝。
花暗省。
许多情、相逢梦境。
便行云、都不归来，也合寄将音信。
孤迥。
盟鸾心在，跨鹤程高，後期无准。
情丝待翦。
翻惹得，...
--------------------------------------------------


In [21]:
import os
import numpy as np
from tokenizers import ByteLevelBPETokenizer

# 输入文件路径
input_file_path = './pretrain.txt'

# 读取原始文本数据
with open(input_file_path, 'r', encoding='utf-8') as f:
    data = f.read()

# 初始化Byte-level BPE分词器
tokenizer = ByteLevelBPETokenizer()

# 训练分词器
tokenizer.train(
    files=[input_file_path],      # 训练文件
    vocab_size=2048,              # 词汇表大小
    min_frequency=2,              # 最小出现频率
    special_tokens=["<|endoftext|>"],  # 特殊标记
)

# 创建保存分词器模型的目录
if not os.path.exists("bbpe"):
    os.mkdir("bbpe")
    
# 保存训练好的分词器模型（词汇表和合并规则）
tokenizer.save_model("bbpe")

['bbpe/vocab.json', 'bbpe/merges.txt']

In [30]:
# 重新加载分词器
tokenizer = ByteLevelBPETokenizer(
    "bbpe/vocab.json",    # 词汇表文件
    "bbpe/merges.txt",    # BPE合并规则文件
)

# 读取文本数据
with open('pretrain.txt', 'r', encoding='utf-8') as f:
    data = f.read()

# 按空行分割成一首首词
poems = data.strip().split('\n\n')

all_token_ids = []

for i, poem in enumerate(poems):
    # 对每首词进行编码
    poem_ids = tokenizer.encode(poem).ids
    
    # 添加到总列表
    all_token_ids.extend(poem_ids)
    all_token_ids.append(0)

# 将ID列表转换为numpy数组，使用uint16类型以节省空间
pretrain_ids = np.array(all_token_ids, dtype=np.uint16)

# 将token ID序列保存为二进制文件
pretrain_ids.tofile('./pretrain.bin')

# 验证一下
print(f"总token数: {len(pretrain_ids)}")
print(f"eostoken数量: {np.sum(pretrain_ids == 0)}")
print(f"诗词数量: {len(poems)}")

# 查看前50个token（包含eostoken）
print("\n前50个token ID:")
print(pretrain_ids[:50])


总token数: 430575
eostoken数量: 4998
诗词数量: 4998

前50个token ID:
[ 926  199  406  315  227  868  839  584  402  326  258  199  528  898
  625 1142 1503  824  258  199  637  971  396  243  541  360  258  199
  507  506 1997  409  258  199 1224 1797  343   97  258  199  603  496
 1957 1247  258  199  700  497  682  961]
